# Goal

## 1.) Implement Integrated Gradients
    a.) Using two versions of Integrated Gradients. The first is a custom implementation adapted from the Authors code provided on their github
    b.) The second is a implementation developed by Naozumi Hiranuma (hiranumn@uw.edu) 
## 2.) Update model to Keras 2.0
    a.) Successfully transferred model to newest keras version

In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
@author: Michael Di Amore
"""

#Had to use python 2 for TF 1.4.0 was having problems with my virutal env
from __future__ import print_function
from __future__ import division

%matplotlib inline
import tensorflow as tf
import Query as query
import quandl 
import numpy as np
# np.random.seed(12345) # Set seed

import pandas as pd
from imblearn.combine import SMOTEENN 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.utils import shuffle
from sklearn.decomposition import PCA


import pdb
from sklearn.metrics import accuracy_score,f1_score,roc_auc_score,recall_score,precision_score
import warnings
import matplotlib.pyplot as plt

import tensorflow as tf
import edward as ed

import keras
import tensorflow.contrib.keras as k


Using TensorFlow backend.


In [2]:
####
#  Load Data. If you don't want to / can't run the query.
# Basically this is all data from 2007-01-01 to 2017-10-06
load_data = True
if load_data == True:
    print ('Loading Data...')
    df_2001_2006 = pd.read_csv('Gdelt_events_20000101_20061231.csv')
    df_2001_2006 = df_2001_2006.set_index('sqldate',drop=True).sort_index()
    df_2007_2013 = pd.read_csv('Gdelt_events_20070101_20131231.csv')
    df_2007_2013 = df_2007_2013.set_index('sqldate',drop=True).sort_index()
    df_2014_01 = pd.read_csv('Gdelt_events_20140101_20140531.csv')
    df_2014_01  = df_2014_01 .set_index('sqldate',drop=True).sort_index()
    df_2014_02 = pd.read_csv('Gdelt_events_20140601_20141231.csv')
    df_2014_02  = df_2014_02 .set_index('sqldate',drop=True).sort_index()
    
    df_2015 = pd.read_csv('Gdelt_events_20150101_20151231.csv')
    df_2015 = df_2015.set_index('sqldate',drop=True).sort_index()
    df_2016_2017 = pd.read_csv('Gdelt_events_20160101_20171006.csv')
    df_2016_2017 = df_2016_2017.set_index('sqldate',drop=True).sort_index()
    gdelt_df = pd.concat([df_2001_2006,df_2007_2013,df_2014_01,df_2014_02,df_2015,df_2016_2017])
    
    del df_2001_2006,df_2007_2013,df_2014_01,df_2014_02,df_2015,df_2016_2017
    
####
# Set Params
lookback_window = int(np.round(252/2))
number_stdev = 3.5
daily_security_vol_target = .15/np.sqrt(252) #.15 is the 10 year SPY volalility found here
#http://performance.morningstar.com/funds/etf/ratings-risk.action?t=SPY&region=usa&culture=en-US
####

proj_id = 'capstone-v0'
start_date = '2000-01-01'
end_date = '2017-10-06'
ticker = '^GSPC'
my_query = query.query_tool(proj_id,start_date,end_date,ticker)
sql_query = """
            SELECT Actor1Name, GoldsteinScale,NumMentions,sourceurl,
            sqldate, avgtone, numarticles, numsources,  
            FROM [gdelt-bq:full.events] 
            WHERE sqldate > 20010101 and sqldate <= 20061231  and 
            Actor1Code like '%BUS%'and
            Actor1Geo_CountryCode like "%US%"
            """
if load_data == False:
    print ('Querying Gdelt...')
    my_query.query_gdelt(sql_query)
    my_query.gdelt_df = my_query.gdelt_df.set_index('sqldate',drop=True).sort_index()
    df = my_query.gdelt_df.copy(True)

#Creating Labels. i.e. if change in spx_return is x standard deviations
security_prices = my_query.query_yahoo()
security_return = np.log(security_prices['Adj Close']).diff() #log Return
security_vol = security_return.rolling(window=lookback_window).std().dropna()
security_mean = security_return.rolling(window=lookback_window).mean().dropna()
security_vol.name = 'Volatility'
security_return = security_return.loc[security_vol.index] #First entry is NAN because of return
# day_over_day_diff = np.abs(security_return.diff())#can subtract because of log returns
# event_idx = [(np.abs(security_mean)+(security_vol * 3.5)) < np.abs(security_return)]
event_idx = [((security_mean + security_vol * number_stdev) < security_return) | ((security_mean - security_vol *number_stdev) > security_return) ]
event_idx = np.array(event_idx).astype(int).flatten()
event_idx = pd.Series(event_idx,index=security_return.index)
print ('Data Loaded')

Loading Data...
Data Loaded


In [4]:
api_key =  ''
wti_co = my_query.query_quandl("FRED/DCOILWTICO",api_key)
unemploy = my_query.query_quandl("FRED/UNEMPLOY",api_key)
m1v = my_query.query_quandl("FRED/M1V",api_key)
m2v = my_query.query_quandl("FRED/M2V",api_key)
stressindex = my_query.query_quandl("FRED/STLFSI", api_key)
dff = my_query.query_quandl("FRED/DFF",api_key)
my_query.set_ticker('^VIX')
vix = my_query.query_yahoo()
vix = vix['Adj Close']
vix.name = 'VIX'
print ('Quandl Loaded')

Quandl Loaded


In [5]:
quandl_others = pd.concat([wti_co,unemploy,m1v,m2v,stressindex,vix,dff],axis=1)
quandl_others.columns = ['wit_co','unemploy','m1v','m2v','slsi','vix','dff']
quandl_others.ffill(inplace=True)
quandl_others.fillna(0,inplace=True)

In [6]:
def normalize_ts(series,lookback=lookback_window):
    rolling_mean = series.rolling(window=lookback).mean()
    rolling_std = series.rolling(window=lookback).std() + .10**3
    normalized = (series-rolling_mean)/rolling_std
    normalized = normalized.dropna()
    return(normalized)

In [7]:
#Collapse numerical data into x,y pairs by taking means
collapsed = gdelt_df.groupby(by=gdelt_df.index).mean()

#Shift data so as only to use yesterday's news for tomorrow's prediction
#i.e. we shift forward, using yesterday data as today
collapsed_shifted = collapsed.shift(1)
quandl_shifted = quandl_others.shift(1)

#Scale the data in such a way that we aren't looking forward into the feature
print ('Scaling using custom scaler')
collapsed_shifted = collapsed_shifted.apply(normalize_ts)
quandl_shifted = quandl_shifted.apply(normalize_ts)

collapsed_shifted.index = pd.to_datetime(collapsed_shifted.index,format='%Y%m%d')
collapsed_shifted = collapsed_shifted.loc[security_vol.index].dropna()
collapsed_shifted = pd.concat([collapsed_shifted,quandl_shifted],axis=1)
event_idx = event_idx.dropna()
collapsed_shifted = collapsed_shifted.loc[event_idx.index]

Scaling using custom scaler


In [8]:
print ('Creating train test split...')
X_train,X_test,Y_train,Y_test = train_test_split(collapsed_shifted,event_idx,stratify=None,test_size=.20,shuffle=False)
X_train,X_val,Y_train,Y_val = train_test_split(X_train,Y_train,test_size=.20,stratify=None,shuffle=False)

#Creating Copies of Data Frames these will be useful later for debugging
X_train_df = X_train.copy(True)
Y_train_df = Y_train.copy(True)
Y_val_df = Y_val.copy(True)
X_train  = np.array(X_train)
X_val = np.array(X_val)
X_test = np.array(X_test)

Creating train test split...


In [9]:

# pca = PCA(whiten=True)
# X_train = pca.fit_transform(X_train)
# X_test = pca.transform(X_test)
# X_val = pca.transform(X_val)

print ('Length of X_train before smote')
print (len(X_train))
print ('Number of Positives before smote')
print (Y_train.sum())


print ('Performing Oversampling/Undersampling...')
s = SMOTEENN()
X_train,Y_train= s.fit_sample(X_train,Y_train)

print ('Length of X_train after smote')
print (len(X_train))

print ('Number of Positive samples after smote')
print (Y_train.sum())

# print ('Proportion after smote {}'.format(sum(Y_train)/len(Y_train))


# X_train = X_train.reshape(-1,X_train.shape[1],1)
# X_test = X_test.reshape(-1,X_test.shape[1],1)
# X_val = X_val.reshape(-1,X_val.shape[1],1)
# print ('Done!')

Length of X_train before smote
2780
Number of Positives before smote
11
Performing Oversampling/Undersampling...
Length of X_train after smote
5479
Number of Positive samples after smote
2769


In [10]:
X_train = X_train.reshape(-1,X_train.shape[1],1)
X_test = X_test.reshape(-1,X_test.shape[1],1)
X_val = X_val.reshape(-1,X_val.shape[1],1)
D = X_train.shape[1]
Y_val = np.array(Y_val)
X_train = X_train.astype(np.float32)
X_val = X_val.astype(np.float32)
Y_train = Y_train.astype(np.int32)
Y_val = Y_val.astype(np.int32)
features = {'x':X_train}
drop_rate = .55


Y_train = Y_train.reshape(-1,1).flatten()
Y_val = Y_val.reshape(-1,1).flatten()
Y_test = np.array(Y_test).reshape(-1,1).flatten()
nh1 = 512.0
nh2 = 256.0
nh3 = 128.0

In [11]:
Y_train.shape

(5479,)

In [12]:
# def tf_model(features,labels,mode): 
tf.reset_default_graph()
data = tf.placeholder(dtype=tf.float32,shape=[None,D,1])
labels = tf.placeholder(dtype=tf.float32,shape=[None,1])
is_training = tf.placeholder(tf.bool)

## Keras Model - 2.0


In [13]:
from keras.layers import Conv1D,Dropout,BatchNormalization,Input,Flatten,MaxoutDense,Dense
from keras.constraints import maxnorm
from keras import backend as K

In [14]:
from keras.constraints import maxnorm
from keras.legacy.layers import MaxoutDense
from keras.layers import Flatten
from keras.layers import MaxPooling1D
from keras.models import Model


def simple_keras_model(original):
    sess = tf.Session(graph=tf.get_default_graph())
    K.set_session(sess)
    
    original_input = Input(shape=(original.shape[1],1),name='orig')
    layer1 =  Conv1D(512, 5, padding='same',activation='relu',kernel_initializer='he_normal',kernel_constraint=maxnorm(0.5))(original_input)
    layer1 = BatchNormalization()(layer1)
    layer1 = Dropout(.55)(layer1)
    
    layer2 = Conv1D(512,5,padding='same',activation='relu',kernel_initializer='he_normal',kernel_constraint=maxnorm(0.5))(layer1)
    layer2 = BatchNormalization()(layer2)
    layer2 = Dropout(.55)(layer2)
    
    layer3 = Conv1D(256,5,padding='same',activation='relu',kernel_initializer='he_normal',kernel_constraint=maxnorm(0.5))(layer2)
    layer3 = BatchNormalization()(layer3)
    layer3 = Dropout(.55)(layer3)

    layer4 = Conv1D(128,5,padding='same',activation='relu',kernel_initializer='he_normal',kernel_constraint=maxnorm(0.5))(layer3)
    layer4 = BatchNormalization()(layer4)
    layer4 = Dropout(.55)(layer4)
    
    
    layer5 = Conv1D(128,3,padding='same',activation='relu',kernel_initializer='he_normal')(layer4)
    layer5 = BatchNormalization()(layer5)
    layer5 = Dropout(.35)(layer5)
    

    layer6 = Conv1D(128,3,padding='same',activation='relu',kernel_initializer='he_normal')(layer5)
    layer6 = BatchNormalization()(layer6)
    layer6 = Dropout(.35)(layer6)
    
    
    layer7 = Conv1D(128,3,padding='same',activation='relu',kernel_initializer='he_normal')(layer6)
    layer7 = BatchNormalization()(layer7)
    layer7 = Dropout(.35)(layer7)
    
    
    layer8 = Conv1D(128,3,padding='same',activation='relu',kernel_initializer='he_normal')(layer7)
    layer8 = BatchNormalization()(layer8)
    layer8 = Dropout(.35)(layer8)
    layer8 = Flatten()(layer8)
    
    layer9 = MaxoutDense(1024,init='he_normal')(layer8)
    layer9 = BatchNormalization()(layer9)
    
    layer10 =MaxoutDense(512,init='he_normal')(layer9)
    layer10 = BatchNormalization()(layer10)
    
    layer11 = MaxoutDense(128,init='he_normal')(layer10)
    layer11 = BatchNormalization()(layer11)
    output = Dense(1,activation='sigmoid')(layer11)
    

    my_model = Model([original_input], output=output)
    optimizer_adam = keras.optimizers.adam(0.001) 
    my_model.compile(loss='binary_crossentropy', optimizer=optimizer_adam, metrics=['accuracy'])


    return(my_model)

In [15]:
my_model = simple_keras_model(X_train)
my_model.fit(X_train,Y_train,epochs=25,batch_size=128)

/home/ubuntu/.local/lib/python3.5/site-packages/keras/legacy/layers.py:527: UserWarning: The `MaxoutDense` layer is deprecated and will be removed after 06/2017.
  warnings.warn('The `MaxoutDense` layer is deprecated '
/home/ubuntu/src/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:61: UserWarning: Update your `Model` call to the Keras 2 API: `Model([<tf.Tenso..., outputs=Tensor("de...)`


Epoch 1/25
5479/5479 [==============================] - 7s - loss: 0.3787 - acc: 0.8500     
Epoch 2/25
5479/5479 [==============================] - 2s - loss: 0.1513 - acc: 0.9496     
Epoch 3/25
5479/5479 [==============================] - 2s - loss: 0.1013 - acc: 0.9655     
Epoch 4/25
5479/5479 [==============================] - 2s - loss: 0.0804 - acc: 0.9735     
Epoch 5/25
5479/5479 [==============================] - 2s - loss: 0.0764 - acc: 0.9752     
Epoch 6/25
5479/5479 [==============================] - 2s - loss: 0.0627 - acc: 0.9810     
Epoch 7/25
5479/5479 [==============================] - 2s - loss: 0.0479 - acc: 0.9836     
Epoch 8/25
5479/5479 [==============================] - 2s - loss: 0.0403 - acc: 0.9892     
Epoch 9/25
5479/5479 [==============================] - 2s - loss: 0.0404 - acc: 0.9881     
Epoch 10/25
5479/5479 [==============================] - 2s - loss: 0.0333 - acc: 0.9885     
Epoch 11/25
5479/5479 [==============================] - 2s - loss: 0

In [16]:
def proba_to_label(pred,threshold):
    ones = np.array([pred > threshold])
    
    ones = ones.flatten()
    return (ones)

In [17]:
probas = my_model.predict(X_val)
predicted_classes = proba_to_label(probas,.50)

In [18]:
acc = accuracy_score(Y_val,predicted_classes)
f1 = f1_score(Y_val,predicted_classes)
roc = roc_auc_score(Y_val,predicted_classes)
prec = precision_score(Y_val,predicted_classes)
recall= recall_score(Y_val,predicted_classes)


print (' Standard Deviations: {}'.format(number_stdev))
print (' Accuracy: {}'.format(acc))
print (' F1: {}'.format(f1))
print (' RoC: {}'.format(roc))
print (' Precision: {}'.format(prec))
print (' Recall: {}'.format(recall))



 Standard Deviations: 3.5
 Accuracy: 0.9237410071942446
 F1: 0.10169491525423728
 RoC: 0.9617052023121387
 Precision: 0.05357142857142857
 Recall: 1.0


# Integrated Gradients 1

Implementation adapted from example codegenerously given by the authors of the paper on their github
#Adjusted from https://github.com/ankurtaly/Integrated-Gradients/blob/master/attributions.ipynb


In [35]:
def T(layer,graph):
    #Helper for getting layer output tensor
    return graph.get_tensor_by_name(layer)

def scorer(sess,tensor,data):
      return sess.run(tensor, {'orig:0':data,is_training:False,K.learning_phase():0})
    
def top_label_and_score(data,graph):
    '''
    Returns the label and score of the object class
    that receives the highest SOFTMAX score.
    '''
    # Evaluate the SOFTMAX output layer for the image and
    # determine the label for the highest-scoring class
    t_softmax = tf.reduce_mean(T('dense_1/Sigmoid:0',graph), reduction_indices=0)
#     t_softmax = T('dense_3/Sigmoid:0',graph)
    scores = scorer(sess, t_softmax, data)
    id = np.argmax(scores)
    return labels[id], scores[id]

def output_label_tensor(label,graph):
    '''Returns a tensor (shape: scalar) representing the SOFTMAX
     for a given label.
    '''
    lab_index = np.where(np.in1d(Y_train, label))[0][0]
    lab_index = lab_index.astype(np.int32)
#     t_softmax = T('dense_3/Sigmoid:0',graph)
    t_softmax = tf.reduce_sum(T('dense_1/Sigmoid:0',graph), reduction_indices=0)

    return t_softmax[lab_index]

def integrated_gradients(data, label,graph, steps=10**6):
    '''
     Returns attributions for the prediction label based
     on integrated gradients at the image.

     Specifically, the method returns the dot product of the image
     and the average of the gradients of the prediction label (w.r.t.
     the image) at uniformly spaced scalings of the provided data.

    '''
    t_output = output_label_tensor(label,graph)  # shape: scalar
    t_grad = tf.gradients(t_output, T('orig:0',graph))[0]
    grads = scorer(sess, t_grad, data)
    return data*np.average(grads, axis=0)


Predict on X_train and get predictions from probas

In [20]:
proba_for_ig = my_model.predict(X_train)
pred_for_ig = proba_to_label(proba_for_ig,.5)

Get session and graph for Integrated Gradients

In [36]:
sess = K.get_session()
sess.run(tf.global_variables_initializer())
ig = integrated_gradients(X_train,pred_for_ig,sess.graph)

In [37]:
total_ig = ig.sum(axis=0)
ig_df = pd.DataFrame(total_ig).T
ig_df.columns = X_train_df.columns

### Results
Integrated Gradients reveal that the GDELT features are not contributing much at all

In [38]:
ig_df.T.sort_values(by=0)

,0
vix,-28.249872
m2v,-8.562857
m1v,-4.975959
slsi,-2.681561
unemploy,-1.719554
wit_co,-1.701710
avgtone,-1.069570
numsources,-0.421932
GoldsteinScale,-0.025769
NumMentions,-0.025184


In [39]:
ig_df.abs().T.sort_values(by=0)

,0
numarticles,0.000829
NumMentions,0.025184
GoldsteinScale,0.025769
numsources,0.421932
dff,0.546937
avgtone,1.069570
wit_co,1.701710
unemploy,1.719554
slsi,2.681561
m1v,4.975959


## Integrated Gradients 2
#### https://github.com/hiranumn/IntegratedGradients/blob/master/IntegratedGradients.py

In [25]:
import IntegratedGradients as ig2 

In [40]:
int_grad = ig2.integrated_gradients(model=my_model)

Evaluated output channel (0-based index): All
Building gradient functions
Progress: 100.0%
Done.


In [45]:
cumulative_sum = np.zeros(X_train.shape[1])
for i in range(len(X_train)):
    cumulative_sum+= int_grad.explain(X_train[i],num_steps=100).flatten()
    if i % 1000 == 0:
        print (i)

0
1000
2000
3000
4000
5000


In [46]:
feature_imp = pd.DataFrame(cumulative_sum).T
feature_imp.columns = X_train_df.columns

In [47]:
feature_imp.T.sort_values(by=0)

,0
vix,-67.333618
m1v,-33.543937
slsi,-30.010685
wit_co,-28.310898
unemploy,-21.436498
m2v,-18.222832
avgtone,-8.722337
dff,-8.638571
numarticles,-8.567603
NumMentions,-6.529546


In [48]:
feature_imp.T.abs().sort_values(by=0)

,0
GoldsteinScale,0.279314
numsources,4.568454
NumMentions,6.529546
numarticles,8.567603
dff,8.638571
avgtone,8.722337
m2v,18.222832
unemploy,21.436498
wit_co,28.310898
slsi,30.010685


## Results 2

The other code reveals a similar story with regard to GDELT, as it's not a heavily contributing set of features. However the order of the financial features differs slightly from my implementation